In [ ]:
# %rm -rf phase2_module
# !git clone https://github.com/LgDNet/phase2_module.git
# !cd phase2_module && make copy

In [1]:
cd ..

/home/elicer


/home/elicer/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from phase2_module.src.utils.dataset import Data
from phase2_module.src.models.classification.models import Model
from phase2_module.src.base import BasePiepline
# from phase2_module.src.hyper_parameters.params_optimization import xgboost_opt, set_params_optimization_data
from phase2_module.src.utils.set_seed import seed_everything
from phase2_module.src.utils.top_score_instance import check_the_score
from phase2_module.src.encoder.encoders import Encode

from phase2_module.src.utils.manage_pkl_files import PickleManager as pkl_manager
from phase2_module.src import preprocess
seed_everything()  # NOTE: default 42
_model = Model()

In [3]:
import pandas as pd
import numpy as np
import re, pickle
import warnings
from tqdm import tqdm
warnings.filterwarnings(action='ignore')
def set_up(mode= None):
    pd.set_option('display.max_columns', mode)
set_up()

# Preprocessing

In [4]:
basic = preprocess.Basic
country = preprocess.Country
customer = preprocess.Customer
inquiry = preprocess.Inquiry
product_category = preprocess.ProductCategory
convert_ratio = preprocess.Convert_ratio
customer2 = preprocess.Customer2

# Main

In [9]:
pkls = pkl_manager.map(pkl_manager.loads, pkl_manager.metadata_directory)

runtime_instance = {
    "Train": Data.train,
    "Test": Data.test,
    "Instances":[
        {"instance": basic, "method": ['drop_duplicated'], "params": False},
        {"instance": country, "method": ['apply'], "params": pkls.get("country")},
        {"instance": inquiry, "method": ["apply"], "params": False},
        {"instance": product_category, "method": ["product_categories", 'fill_odds'], "params": {**pkls.get("product_category"), **pkls.get("product_subcategory")}},
        {"instance": customer, "method": ["apply"], "params": pkls.get("customer")},
        {"instance": convert_ratio, "method": ["lead_owner","customer_idx",'lead_owner_customer_idx_means','convert_all_true'], "params": {**pkls.get("lead_owner"), **pkls.get("customer_idx")}},
        {"instance": basic, "method": ['cusotmer_idx_categorization','customer_idx_merge_enterprise'], "params": False},
        {"instance": customer2, "method": ['seniority_level'], "params": pkls.get("customer2")},
    ]
}

## preprocessing

In [10]:
df_train = preprocess.runner('Train',**runtime_instance)
df_test = preprocess.runner('Test',**runtime_instance)

In [14]:
# # prev column도 영향이 그렇게 많지는 않다. 뺴고 historical 그대로 사용하는 방법..?

# cond = df_train['historical_existing_cnt']> 0 
# df_train.loc[cond,"prev"] = 1
# df_train['prev'] = df_train['prev'].fillna(0)

# cond = df_test['historical_existing_cnt']> 0 
# df_test.loc[cond,"prev"] = 1
# df_test['prev'] = df_test['prev'].fillna(0)

In [15]:
# # lead_owner_converted_ratio 구간화, 별로 안좋은 것 같다.

# # bin = [0.0001,10.0001,20.0001,30.0001,40.0001,50.0001,60.0001,70.0001,80.0001,90.0001,100.00,100.001]
# bin = [0.0001,10.0001,30.0001,50.0001,70.0001,90.0001,100.00,100.001]
# df_train.loc[:,'lead_owner_converted_ratio'] = pd.cut(df_train['lead_owner_converted_ratio'], bins=bin, right = False,labels= False)
# df_train['lead_owner_converted_ratio'] = df_train['lead_owner_converted_ratio']+1
# df_train['lead_owner_converted_ratio'].fillna(0.0,axis = 0, inplace  = True)
# # bin = [0.0001,10.0001,20.0001,30.0001,40.0001,50.0001,60.0001,70.0001,80.0001,90.0001,100.00,100.001]
# bin = [0.0001,10.0001,30.0001,50.0001,70.0001,90.0001,100.00,100.001]
# df_test.loc[:,'lead_owner_converted_ratio'] = pd.cut(df_test['lead_owner_converted_ratio'], bins=bin, right = False,labels= False)
# df_test['lead_owner_converted_ratio'] = df_test['lead_owner_converted_ratio']+1
# df_test['lead_owner_converted_ratio'].fillna(0.0,axis = 0, inplace  = True)

## encoding

In [16]:
encode = Encode()

In [17]:
exisiting_columns = [
                     'business_area','business_subarea','business_unit',
                        'enterprise',
#                         'response_corporate',"customer_country", # country
#                         'customer_type','customer_job','customer_position', #customer
#                         'inquiry_type','expected_timeline', # inquiry
#                         'product_subcategory','product_category','product_modelname' # product_category
                       ] 
preprocess_columns = [
                     'country','continent', # country
                     'customer_type2','job_function','seniority_level', # customer
                     'new_expected_timeline', 'new_inquiry_type',# inquiry
                     'category_2','category_3','category_1', # product_category
#                      'test2'
                     ]

label_encode_columns = exisiting_columns + preprocess_columns

In [18]:
# 결측치 채우기
df_train = preprocess.fillna(df_train)
df_test = preprocess.fillna(df_test)

In [19]:
# # one-hot encoding
# onehot_encode_columns = ['category_1']
# df_train = encode.onehot_encoder(df_train,onehot_encode_columns)
# df_test = encode.onehot_encoder(df_test,onehot_encode_columns)

In [20]:
# label encoding
df_train = encode.label_encoder(df_train,label_encode_columns)
df_test = encode.label_encoder(df_test,label_encode_columns)

# Modeling

## drop column
모델링 전이 column을 drop하면서 다루기가 쉽다고 판단하였음

In [21]:
drop_col = [
            'id_strategic_ver','it_strategic_ver','idit_strategic_ver','ver_pro',
            'customer_country.1',
            'response_corporate',"customer_country", # country
            'customer_type','customer_job','customer_position', # customer
            'inquiry_type', 'expected_timeline', #inquiry
            'product_subcategory','product_category','product_modelname', # product_category
#             'cate_is_nan', # product_category
    
#             'com_reg_ver_win_rate','idit_strategic_ver','ver_cus',
#             'ver_pro','ver_win_rate_x','ver_win_ratio_per_bu',
            'customer_idx',
            'customer_idx_converted_ratio', # ratio
            'new_expected_timeline'
            #'business_unit',
#             'historical_existing_cnt',
            #'lead_owner', 'lead_owner_converted_ratio', 'lead_owner_customer_idx_mean', # lead_owner
            ]
df_train.drop(drop_col,axis = 1, inplace = True)
df_test.drop(drop_col,axis = 1, inplace = True)

## Scaler

In [22]:
import numpy as np
# log 변환
df_train['lead_desc_length'] = np.log1p(df_train['lead_desc_length'])
df_test['lead_desc_length'] = np.log1p(df_test['lead_desc_length'])

In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train[['lead_owner_converted_ratio','lead_owner_customer_idx_mean']] = scaler.fit_transform(df_train[['lead_owner_converted_ratio','lead_owner_customer_idx_mean']])
df_test[['lead_owner_converted_ratio','lead_owner_customer_idx_mean']] = scaler.transform(df_test[['lead_owner_converted_ratio','lead_owner_customer_idx_mean']])

## train

In [24]:
dict_ = {'criterion': 'log_loss', 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 2, 'random_state': 42}
model = _model.decision_tree
model.set_params(**dict_)

DecisionTreeClassifier(criterion='log_loss', max_depth=20, min_samples_leaf=2,
                       min_samples_split=7, random_state=42)

In [25]:
# model = _model.xgboost
# model.set_params(random_state =42)

In [26]:
X = df_train.drop(["is_converted"],axis = 1)
Y = df_train['is_converted']

In [27]:
from sklearn.model_selection import StratifiedKFold
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
result = {"f1":[], "precision":[], "recall":[],'roc_auc_score':[]}
train_result = {"f1":[], "precision":[], "recall":[],'roc_auc_score':[]}
models = [] # 모델을 저장할 리스트

# k-fold
for train_idx, test_idx in tqdm(stratkfold.split(X,Y)):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

    # 모델 훈련
    model.fit(x_train, y_train)
    models.append(model) # 훈련된 모델을 리스트에 추가
    train_pred = model.predict(x_train)
    predict = model.predict(x_test)

    score_result = check_the_score(train_pred, y_train)
    for name, score in score_result.items():
        train_result[name].append(score)

    score_result = check_the_score(predict, y_test)
    for name, score in score_result.items():
        result[name].append(score)

print('----[K-Fold Train Score]-----')
for name, score_list in train_result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

print('----[K-Fold Validation Score]-----')
for name, score_list in result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

5it [00:00,  6.18it/s]

----[K-Fold Train Score]-----
f1 score : 0.9906 / STD: (+/- 0.0006)
precision score : 0.9873 / STD: (+/- 0.0016)
recall score : 0.9938 / STD: (+/- 0.0015)
roc_auc_score score : 0.9964 / STD: (+/- 0.0007)
----[K-Fold Validation Score]-----
f1 score : 0.9729 / STD: (+/- 0.0018)
precision score : 0.9706 / STD: (+/- 0.0046)
recall score : 0.9752 / STD: (+/- 0.0028)
roc_auc_score score : 0.9863 / STD: (+/- 0.0013)


In [28]:
feature_importances = model.feature_importances_
feature_names = x_train.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

convert_all_true: 0.47939585903705617
enterprise: 0.30062112616379855
customer_idx_group: 0.13683121305265583
lead_owner_converted_ratio: 0.05194450236791924
historical_existing_cnt: 0.006775920751503432
lead_desc_length: 0.005680259699628416
customer_type2: 0.0045151077008649294
lead_owner: 0.0037165872352531793
country: 0.0019335939093374586
bant_submit: 0.0017877107736668052
job_function: 0.0009763263659493259
ver_win_rate_x: 0.0008269351195158931
category_1: 0.0007492342707613597
business_subarea: 0.0007315662400073251
customer_interest: 0.0007024568452354531
ver_win_ratio_per_bu: 0.0006948290936560926
category_3: 0.0006822550206611132
lead_owner_customer_idx_mean: 0.0004482423475191079
new_inquiry_type: 0.0002983959920866073
category_2: 0.0002114426078341586
less_timeline: 0.0002027985742583001
business_area: 0.00012180515240499469
ver_cus: 7.885473994300878e-05
seniority_level: 7.297693848325198e-05
business_unit: 0.0
com_reg_ver_win_rate: 0.0
continent: 0.0


## test & submission

In [29]:
test_drop = df_test.drop(['id','is_converted'],axis = 1)

In [30]:
pred = model.predict(test_drop)
pred = np.where(pred == 0, False, True)

In [31]:
# final_predictions = np.array([model.predict(test_drop) for model in models])
# final_prediction = np.round(final_predictions.mean(axis=0))
# pred = np.where(final_prediction == 0, False, True)

In [32]:
sum(pred)

283

In [ ]:
pred2 = pred  #pred1 1000 | pred2 919 | pred3 820

In [ ]:
su = 0
for i,j in zip(pred2, pred1):
    if i != j:
        su +=1
su

In [ ]:
len(df_test)

In [ ]:
pred1_ = pred1.reshape(-1,1)
pred2_ = pred2.reshape(-1,1)
pred3_ = pred3.reshape(-1,1)
combine = np.hstack((pred1_, pred2_, pred3_))
combine_ = np.round(combine.mean(axis=1))

In [ ]:
pred = np.where(pred == 0, False, True)
df_test['is_converted'] = pred
df_test.to_csv("submission.csv",index = False)